### Import packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install av
import torch, torchvision
import torchvision.transforms as transforms
import cv2
import os
from os import path
import json

### Extract frames from videos and place them inside video name folder

In [ ]:
# def extract_frames(file_path, target_dir):
#     if not os.path.exists(target_dir):
#         os.makedirs(target_dir)

#     vidcap = cv2.VideoCapture(file_path)
#     success, image = vidcap.read()
#     count = 0
 
#     while success:
#       frame_path = os.path.join(target_dir, f'{count}.jpg')
#       cv2.imwrite(frame_path, image)     # save frame as JPEG file
#       count += 1
#       success, image = vidcap.read()
#       success, image = vidcap.read()
#       success, image = vidcap.read()


# root = '/content/drive/MyDrive/mini_dataset'
# frame_root = '/content/drive/MyDrive/teste/hmdb51_frames'
# labels_path = '/content/drive/MyDrive/teste/labels.csv'
# class_name_to_label_path = '/content/drive/MyDrive/teste/class_name_to_label.json'

# # read files
# files = []

# for class_name in os.listdir(root):
#     for video_name in os.listdir(os.path.join(root, class_name)):
#         files.append([os.path.join(class_name, video_name), class_name])

# # normalize labels
# class_name_to_label = {}
# current_label = -1

# for vid in files:
#     label = class_name_to_label.get(vid[1], -1)

#     if label == -1:
#         current_label += 1
#         class_name_to_label[vid[1]] = current_label
#         label = current_label

#     vid[1] = label


# # save file paths
# if not os.path.exists(os.path.split(labels_path)[0]):
#     os.makedirs(os.path.split(labels_path)[0])

# f = open(labels_path, 'w')

# f.write('path,label\n')

# for vid in files:
#     f.write(f'{vid[0]},{vid[1]}\n')

# f.close()

# # save label normalization
# if not os.path.exists(os.path.split(class_name_to_label_path)[0]):
#     os.makedirs(os.path.split(class_name_to_label_path)[0])

# with open(class_name_to_label_path, 'w') as json_file:
#     json.dump(class_name_to_label, json_file, indent=4)

# # extract frames
# for i, vid in enumerate(files):
#     file_path = os.path.join(root, vid[0])
#     target_dir = os.path.join(frame_root, vid[0])

#     extract_frames(file_path, target_dir)

#     print(f'{i+1}/{len(files)}')


### Split labels csv file into train and validate - 80/20

In [ ]:
# import pandas as pd

# def split(label):
#     path = '/content/drive/MyDrive/teste'
#     train = label.sample(frac=0.8, random_state=201)
#     val = label.drop(train.index)
#     train.to_csv(path + '/train.csv', mode='a', header=False)
#     val.to_csv(path + '/val.csv', mode='a', header=False)


# root = '/content/drive/MyDrive/teste'
# frame_root = '/content/drive/MyDrive/teste/hmdb51_frames'
# labels_path = '/content/drive/MyDrive/teste/labels.csv'
# class_name_to_label_path = '/content/drive/MyDrive/teste/class_name_to_label.json'

# labels_path = '/content/drive/MyDrive/teste/labels.csv'
# labels_data = pd.read_csv(labels_path)
# last_label = labels_data.tail(1)['label']
# last_label = last_label.iloc[0]
# max_labels = labels_data.nunique()
# max_labels= max_labels['label']
# index = 0

# for i in range(max_labels):
#     label = labels_data[labels_data['label'] == index]
#     split(label)
#     index +=1 


### Load the images to a model

In [34]:
import pandas as pd
from PIL import Image
import random
import torchvision.transforms.functional as TF

import torch
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
import matplotlib.pyplot as plt

train_csv_path = '/content/drive/MyDrive/teste/train.csv'
validate_csv_path = '/content/drive/MyDrive/teste/val.csv'
root_path = '/content/drive/MyDrive/teste/hmdb51_frames'

class MyDataset(Dataset):
  def __init__(self, frames_csv_file, root_dir):
    self.frames_csv = pd.read_csv(frames_csv_file)
    self.root_dir = root_dir
    self.slice_size = 10

  def __len__(self):
      return len(self.frames_csv)

  def __getitem__(self, idx):
      if torch.is_tensor(idx):
          idx = idx.tolist()
      
      #need to take 10 frames and stack them on 1                                                    
                                                                            #second index has to be 1 because id column (3 columns) 
                                                                                                        #why have to be NOT??? is wrong!! but works..
      all_frames = [f for f in os.listdir(os.path.join(self.root_dir, str(self.frames_csv.iloc[idx, 1]))) if not os.path.isfile(f)]
      images = []
      for frame in all_frames:
        path = os.path.join(self.root_dir, str(self.frames_csv.iloc[idx, 1])) + '/' + frame
        images.append(Image.open(path).convert('RGB'))
      #random get 10 consecutive frames from all video images
      if len(images) > (self.slice_size - 1):
        start = random.randrange(len(images) - 10)
        frames = images[start: start + self.slice_size]
      #tranform all the frames in 1 tensor
      final_image = self.transform(frames)
      tag = int(self.frames_csv.iloc[idx, 2])
      sample = {'image': final_image, 'tag': tag}
      return sample

  def transform(self, frames):
    #random crop
    #random horizontal flip
    #transform to tensor and stack
    frames_tensors = []
    for frame in frames:
      #rezise image
      newsize = (240, 320)
      fame = frame.resize(newsize)
      frame = TF.to_tensor(frame)
      frames_tensors.append(frame)
    final_tensor = torch.stack(frames_tensors, dim=1)
    #normalize

    return final_tensor

train_dataset = MyDataset(train_csv_path, root_path)
validate_dataset = MyDataset(validate_csv_path, root_path)



## Dataloaders

In [41]:
from torch.utils.data import DataLoader

#train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
validate_dataloader = DataLoader(validate_dataset, batch_size=1, shuffle=True)

sample = iter(validate_dataloader).next()
print('tensor: ', end='  ')
print(sample['image'])
print('tag: ', end='  ')
print(sample['tag'])


tensor:   tensor([[[[[0.0039, 0.0039, 0.0039,  ..., 0.3255, 0.3255, 0.3255],
           [0.0039, 0.0039, 0.0039,  ..., 0.3333, 0.3333, 0.3333],
           [0.0039, 0.0039, 0.0039,  ..., 0.3412, 0.3451, 0.3490],
           ...,
           [0.3216, 0.3216, 0.3255,  ..., 0.2275, 0.2353, 0.2392],
           [0.3176, 0.3216, 0.3294,  ..., 0.2275, 0.2353, 0.2353],
           [0.3137, 0.3216, 0.3333,  ..., 0.2196, 0.2275, 0.2275]],

          [[0.0039, 0.0039, 0.0039,  ..., 0.3255, 0.3255, 0.3255],
           [0.0039, 0.0039, 0.0039,  ..., 0.3333, 0.3333, 0.3333],
           [0.0000, 0.0000, 0.0000,  ..., 0.3412, 0.3451, 0.3490],
           ...,
           [0.3020, 0.3020, 0.3020,  ..., 0.2510, 0.2588, 0.2588],
           [0.3176, 0.3216, 0.3294,  ..., 0.2471, 0.2510, 0.2471],
           [0.3137, 0.3255, 0.3333,  ..., 0.2392, 0.2392, 0.2314]],

          [[0.0039, 0.0039, 0.0039,  ..., 0.3255, 0.3255, 0.3216],
           [0.0039, 0.0039, 0.0039,  ..., 0.3333, 0.3333, 0.3294],
           [0.00